In [2]:
import requests
import speech_recognition as sr

GROQ_API_KEY = "--------------Your API KEY--------------"

def ai_giving_response(user_input):
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "llama-3.3-70b-versatile",  # Ensure it's a supported model
        "messages": [{"role": "user", "content": user_input}]
    }
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        response_json = response.json()
        if "choices" in response_json and len(response_json["choices"]) > 0:
            return response_json["choices"][0]["message"]["content"]
        else:
            return "I received an unexpected response."
    else:
        print("Error:", response.text)
        return "I encountered an error."

user_input = "Hello! What can you do?"
response = ai_giving_response(user_input)
print(response)

Error: {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jqfxbsz2eh2s333nq0vjdrmw` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 100017, Requested 10. Please try again in 23.619999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}

I encountered an error.


In [ ]:
from autogen import AssistantAgent
import time
import speech_recognition as sr
#----------------------------------------------SPLIT------------------------SPLIT-----------------SPLIT---------------------
# Define the AI assistants without memory_config
planner = AssistantAgent(
    name="PlannerAgent",
    llm_config={
        "api_key": "--------------Your API KEY--------------",  
        "base_url": "https://api.groq.com/openai/v1",
        "model": "llama-3.3-70b-versatile"
    }
)

executor = AssistantAgent(
    name="ExecutorAgent",
    llm_config={
        "api_key": "--------------Your API KEY--------------",  
        "base_url": "https://api.groq.com/openai/v1",
        "model": "llama-3.3-70b-versatile"
    }
)

decision_maker = AssistantAgent(
    name="DecisionAgent",
    llm_config={
        "api_key": "--------------Your API KEY--------------",  
        "base_url": "https://api.groq.com/openai/v1",
        "model": "llama-3.3-70b-versatile"
    }
)

perceptor = AssistantAgent(
    name="PerceptionAgent",
    llm_config={
        "api_key": "--------------Your API KEY--------------",  
        "base_url": "https://api.groq.com/openai/v1",
        "model": "llama-3.3-70b-versatile"
    }
)
# Including a new summariser agent
summariser = AssistantAgent(
    name="Summariser",
    llm_config={
        "api_key": "--------------Your API KEY--------------",  
        "base_url": "https://api.groq.com/openai/v1",
        "model": "llama-3.3-70b-versatile"
    }
)

# Global message history to simulate memory
history = {
    "PlannerAgent": [],
    "ExecutorAgent": [],
    "DecisionAgent": [],
    "PerceptionAgent": []
}
#----------------------------------------------SPLIT------------------------SPLIT-----------------SPLIT---------------------
# Define a function to log and store messages in history
def log_and_store(agent_name, recipient, message):
    if message:  # Check if the message is not empty or None
        history[agent_name].append({"to": recipient, "message": message})
        print(f"\n{agent_name} → {recipient}: {message}\n")
    else:
        print(f"Error: {agent_name} failed to send message to {recipient}.")

# Define the enhanced workflow with memory integration
def multi_agent_task_with_feedback():
    print("Starting enhanced multi-agent task with memory...\n")

    # Step 1: Perceptor → Planner
    try:
        perceptor_response = perceptor.initiate_chat(
            recipient=planner,
            message="The Robot detects that a cliff is coming up in its path. What are the actions it should take to not pummel itself to death?"
        )
        log_and_store(perceptor.name, planner.name, perceptor_response)
    except Exception as e:
        print(f"Error in Step 1: {e}")
        time.sleep(2)  # Retry after a brief pause
#----------------------------------------------SPLIT------------------------SPLIT-----------------SPLIT---------------------
    # Step 2: Planner → Executor
    try:
        planner_response = planner.initiate_chat(
            recipient=executor,
            message=perceptor_response 
        )
        log_and_store(planner.name, executor.name, planner_response)
    except Exception as e:
        print(f"Error in Step 2: {e}")
        time.sleep(2)
#----------------------------------------------SPLIT------------------------SPLIT-----------------SPLIT---------------------
    # Step 3: Executor → DecisionAgent
    try:
        executor_response = executor.initiate_chat(
            recipient=decision_maker,
            message=planner_response
        )
        log_and_store(executor.name, decision_maker.name, executor_response)
    except Exception as e:
        print(f"Error in Step 3: {e}")
        time.sleep(2)
#----------------------------------------------SPLIT------------------------SPLIT-----------------SPLIT---------------------
    # Step 4: DecisionAgent → Planner (feedback)
    try:
        feedback = decision_maker.initiate_chat(
            recipient=planner,
            message=executor_response  
        )
        log_and_store(decision_maker.name, planner.name, feedback)
    except Exception as e:
        print(f"Error in Step 4: {e}")
        time.sleep(2)
#----------------------------------------------SPLIT------------------------SPLIT-----------------SPLIT---------------------
    # Step 5: Planner uses feedback to re-plan
    try:
        refined_plan = planner.initiate_chat(
            recipient=executor,
            message="Thanks for the feedback!"
        )
        log_and_store(planner.name, executor.name, refined_plan)
    except Exception as e:
        print(f"Error in Step 5: {e}")
        time.sleep(2)
#----------------------------------------------SPLIT------------------------SPLIT-----------------SPLIT---------------------   
    # Step 6: Summariser of the whole ass thing!!!!
    try:
        summary_prompt = (
            "Summarize the entire task flow above, including observations, plans, execution, decisions,"
            "and feedback in a concise mission debrief."
        )
        summary_response = summariser.initiate_chat(
            recipient=None,
            message=summary_prompt
        )
        if summary_response:
            log_and_store(summariser.name, "All Agents", summary_response)
        else:
            print("Summariser didn't return any output (maybe due to API limit or prior failures).")
    except Exception as e:
        print(f"Error in Step 6: {e}")
        time.sleep(2)
    

    print("\nConversation History:")
    for agent, messages in history.items():
        print(f"\n{agent} history:")
        for msg in messages:
            print(f"  To {msg['to']}: {msg['message']}")

if __name__ == "__main__":
    for _ in range(2):
        multi_agent_task_with_feedback()


C:\Users\hankp\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


Starting enhanced multi-agent task with memory...

PerceptionAgent (to PlannerAgent):

The Robot detects that a cliff is coming up in its path. What are the actions it should take to not pummel itself to death?

--------------------------------------------------------------------------------
[autogen.oai.client: 04-26 07:08:35] {695} WARNING - Model llama-3.3-70b-versatile is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
PlannerAgent (to PerceptionAgent):

To prevent the robot from going over the cliff, it should take the following actions:

1. **Stop immediately**: The robot should stop its movement to avoid going over the cliff. This can be achieved by sending a stop command to its motors.

2. **Reversing direction**: After stopping, the robot can reverse its direction to move away from the cliff. This can be done by sending a reverse command to its motors.

3. **Changing route**: